In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from qiskit_qec.circuits import ArcCircuit

links = [(0, 1, 2), (2, 3, 4), (4, 5, 6), (6, 7, 8), (8, 9, 10), (10, 11, 12), (0, 14, 18), (4, 15, 22), (8, 16, 26), (12, 17, 30), (18, 19, 20), (20, 21, 22), (22, 23, 24), (24, 25, 26), (26, 27, 28), (28, 29, 30), (30, 31, 32), (20, 33, 39), (24, 34, 43), (28, 35, 47), (32, 36, 51), (37, 38, 39), (39, 40, 41), (41, 42, 43), (43, 44, 45), (45, 46, 47), (47, 48, 49), (49, 50, 51), (37, 52, 56), (41, 53, 60), (45, 54, 64), (49, 55, 68), (56, 57, 58), (58, 59, 60), (60, 61, 62), (62, 63, 64), (64, 65, 66), (66, 67, 68), (68, 69, 70), (58, 71, 77), (62, 72, 81), (66, 73, 85), (70, 74, 89), (75, 76, 77), (77, 78, 79), (79, 80, 81), (81, 82, 83), (83, 84, 85), (85, 86, 87), (87, 88, 89), (75, 90, 94), (79, 91, 98), (83, 92, 102), (87, 93, 106), (94, 95, 96), (96, 97, 98), (98, 99, 100), (100, 101, 102), (102, 103, 104), (104, 105, 106), (106, 107, 108), (96, 109, 114), (100, 110, 118), (104, 111, 122), (108, 112, 126), (114, 115, 116), (116, 117, 118), (118, 119, 120), (120, 121, 122), (122, 123, 124), (124, 125, 126)]
T = 20
schedule = [[(0, 14), (2, 3), (4, 15), (6, 7), (8, 16), (10, 11), (12, 17), (18, 19), (22, 23), (26, 27), (30, 31), (20, 33), (24, 34), (28, 35), (32, 36), (39, 40), (43, 44), (47, 48), (37, 52), (41, 53), (45, 54), (49, 55), (56, 57), (60, 61), (64, 65), (68, 69), (58, 71), (62, 72), (66, 73), (70, 74), (77, 78), (81, 82), (85, 86), (75, 90), (79, 91), (83, 92), (87, 93), (94, 95), (98, 99), (102, 103), (106, 107), (96, 109), (100, 110), (104, 111), (108, 112), (114, 115), (118, 119), (122, 123)], [(0, 1), (4, 5), (8, 9), (18, 14), (22, 15), (26, 16), (30, 17), (20, 21), (24, 25), (28, 29), (39, 33), (43, 34), (47, 35), (51, 36), (37, 38), (41, 42), (45, 46), (49, 50), (56, 52), (60, 53), (64, 54), (68, 55), (58, 59), (62, 63), (66, 67), (77, 71), (81, 72), (85, 73), (89, 74), (75, 76), (79, 80), (83, 84), (87, 88), (94, 90), (98, 91), (102, 92), (106, 93), (96, 97), (100, 101), (104, 105), (114, 109), (118, 110), (122, 111), (126, 112), (116, 117), (120, 121), (124, 125)], [(2, 1), (4, 3), (6, 5), (8, 7), (10, 9), (12, 11), (22, 21), (26, 25), (30, 29), (20, 19), (24, 23), (28, 27), (32, 31), (39, 38), (43, 42), (47, 46), (51, 50), (41, 40), (45, 44), (49, 48), (60, 59), (64, 63), (68, 67), (58, 57), (62, 61), (66, 65), (70, 69), (77, 76), (81, 80), (85, 84), (89, 88), (79, 78), (83, 82), (87, 86), (98, 97), (102, 101), (106, 105), (96, 95), (100, 99), (104, 103), (108, 107), (118, 117), (122, 121), (126, 125), (116, 115), (120, 119), (124, 123)]]
resets = False
run_202 = False
basis = 'zx'
color = {0: 0, 2: 1, 4: 0, 6: 1, 8: 0, 10: 1, 12: 0, 18: 1, 22: 1, 26: 1, 30: 1, 20: 0, 24: 0, 28: 0, 32: 0, 39: 1, 43: 1, 47: 1, 51: 1, 37: 0, 41: 0, 45: 0, 49: 0, 56: 1, 60: 1, 64: 1, 68: 1, 58: 0, 62: 0, 66: 0, 70: 0, 77: 1, 81: 1, 85: 1, 89: 1, 75: 0, 79: 0, 83: 0, 87: 0, 94: 1, 98: 1, 102: 1, 106: 1, 96: 0, 100: 0, 104: 0, 108: 0, 114: 1, 118: 1, 122: 1, 126: 1, 116: 0, 120: 0, 124: 0}

code = {}
for logical in ['0', '1']:
    code[logical] = ArcCircuit(links, T, schedule=schedule, logical=logical, resets=resets, run_202=run_202, basis=basis, color=color)

In [4]:
from qiskit_qec.decoders import UnionFindDecoder
dec = UnionFindDecoder(code['0'], use_peeling=False)

# Soft Reweighting

In [10]:
from typing import List
from copy import deepcopy
import warnings

import numpy as np

from soft_info import llh_ratio

def soft_reweight(decoder, IQ_data, kde_dict: dict, scaler_dict : dict, layout : List[int], p_data : float = None, p_meas : float = None, common_measure = None):
    
    graph = deepcopy(decoder.decoding_graph.graph)

    p_data = p_data if p_data is not None else 6.836e-3  # Sherbrooke median
    p_meas = p_meas if p_meas is not None else 0

    tot_nb_checks = decoder.code.d - 1  # hardcoded for RepetitionCode

    if layout is not None:
        qubit_mapping = get_repcode_IQ_map(layout, decoder.code.T) #TODO(2) ADAPT TO ARCCIRCUIT

    for edge_idx, edge in enumerate(graph.edges()):

        if edge.qubits is not None: # data edges
            edge.weight = -np.log(p_data/(1-p_data))
        else: # time edges
            edge.weight = 0  #TODO -np.log(p_meas/(1-p_meas))

        src_node_idx, tgt_node_idx = graph.edge_list()[edge_idx]
        src_node, tgt_node = graph.nodes()[src_node_idx], graph.nodes()[
            tgt_node_idx]
        time_source, time_tgt = src_node['time'], tgt_node['time']

        if time_source is None or time_tgt is None:
            continue # skip boundary edges

        if time_source > time_tgt: #TODO check if needed 
            src_node, tgt_node = tgt_node, src_node
            time_source, time_tgt = time_tgt, time_source
            src_node_idx, tgt_node_idx = tgt_node_idx, src_node_idx
            warnings.warn("time_source > time_tgt. Reordering them...") # For debugging purposes.

        if time_source != time_tgt: # time edges or mixed edges
            link_qubit_number = src_node.index 
            IQ_point = IQ_data[time_source * tot_nb_checks + link_qubit_number]

            #TODO Wrong qubit into qubit_mapping?
            layout_qubit_idx = qubit_mapping[src_node_idx]
            kde_0, kde_1 = kde_dict.get(layout_qubit_idx, (None, None))
            scaler = scaler_dict.get(layout_qubit_idx, None)

            weight = llh_ratio(IQ_point, kde_0, kde_1, scaler)

            edge.weight += weight

        #Round the weights to common measure
        if common_measure is not None:
            edge.weight = round( edge.weight/ common_measure) * common_measure

    return graph

# Decoding graph testing

In [13]:
dec.decoding_graph.graph.nodes()

[{'is_boundary': True, 'time': None, 'qubits': [0], 'index': 0, 'properties': {'conjugate': False, 'link qubit': None}},
 {'is_boundary': False, 'time': 0, 'qubits': [124, 126], 'index': 0, 'properties': {'conjugate': False, 'link qubit': 125}},
 {'is_boundary': False, 'time': 1, 'qubits': [124, 126], 'index': 0, 'properties': {'conjugate': False, 'link qubit': 125}},
 {'is_boundary': False, 'time': 2, 'qubits': [124, 126], 'index': 0, 'properties': {'conjugate': False, 'link qubit': 125}},
 {'is_boundary': False, 'time': 3, 'qubits': [124, 126], 'index': 0, 'properties': {'conjugate': False, 'link qubit': 125}},
 {'is_boundary': False, 'time': 4, 'qubits': [124, 126], 'index': 0, 'properties': {'conjugate': False, 'link qubit': 125}},
 {'is_boundary': False, 'time': 5, 'qubits': [124, 126], 'index': 0, 'properties': {'conjugate': False, 'link qubit': 125}},
 {'is_boundary': False, 'time': 6, 'qubits': [124, 126], 'index': 0, 'properties': {'conjugate': False, 'link qubit': 125}},
 {'i